<a href="https://colab.research.google.com/github/nicolez9911/colab/blob/main/02_ExerciseCatsandDogs-0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cats VS Dogs

Install latest version of kaggle library

In [37]:
!pip install --upgrade --force-reinstall --no-deps kaggle

  Using cached kaggle-1.5.16-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16


1. Create a kaggle account and from your settings create a new API token to download a file called kaggle.json

In [38]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nicolez9911","key":"439a3da38e3dab16713f10b91119c5cf"}'}

In [39]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [40]:
!ls ~/.kaggle

kaggle.json


2. Go to this link :https://www.kaggle.com/c/dogs-vs-cats/data
and accept the terms of the competition

In [41]:
!kaggle competitions download -c dogs-vs-cats

100% 810M/812M [00:37<00:00, 24.2MB/s]
100% 812M/812M [00:37<00:00, 22.5MB/s]


In [47]:
!unzip -qq /content/dogs-vs-cats.zip

replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [48]:
!ls /content

cats_vs_dogs_small  kaggle.json  sampleSubmission.csv  train.zip
dogs-vs-cats.zip    sample_data  test1.zip


In [49]:
!unzip -qq /content/train.zip
!unzip -qq /content/test1.zip

**Copying images to training, validation, and test directories**

In [52]:
import os, shutil, pathlib

original_dir = pathlib.Path("/content/train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

### Load Dataset

In [53]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Found 2000 files belonging to 3 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


# build the model

In [54]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(180, 180, 3)),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset, epochs=8)

test_loss = model.evaluate(test_dataset)
validation_loss = model.evaluate(validation_dataset)

test_loss, validation_loss

Epoch 1/8
63/63 [==============================] - 6s 62ms/step - loss: 237.9688 - accuracy: 0.5155
Epoch 2/8
63/63 [==============================] - 4s 62ms/step - loss: 0.7386 - accuracy: 0.6485
Epoch 3/8
63/63 [==============================] - 5s 73ms/step - loss: 0.4320 - accuracy: 0.7945
Epoch 4/8
63/63 [==============================] - 4s 62ms/step - loss: 0.2608 - accuracy: 0.9045
Epoch 5/8
63/63 [==============================] - 4s 62ms/step - loss: 0.1289 - accuracy: 0.9595
Epoch 6/8
63/63 [==============================] - 5s 72ms/step - loss: 0.0678 - accuracy: 0.9850
Epoch 7/8
63/63 [==============================] - 4s 62ms/step - loss: 0.0417 - accuracy: 0.9915
Epoch 8/8
32/32 [==============================] - 1s 31ms/step - loss: 67.6489 - accuracy: 0.1390


([66.58799743652344, 0.1535000056028366],
 [67.6489486694336, 0.13899999856948853])

In [55]:
#!rm -rf cats_vs_dogs_small

In [56]:
#!rm -rf test1

In [57]:
#!rm -rf train